# Домашнее задание №1: Классификация. 
# Логистическая регрессия и kNN.

Форма для сдачи будет опубликована позднее.

### Пожалуйста, не производите никаких манипуляций с  переменной `_dict_for_submission` и с ячейками, в которых она используется. На ее основе будет сгенерирован итоговый файл для сдачи задания.

In [383]:
# Do not change me, just run
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
_dict_for_submission = dict()


def transform_list(list_of_ints):
    return [str(x) for x in list_of_ints]
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

### Часть 1. Небольшой numpy-разогрев. (10%)

In [384]:
import numpy as np
import pandas as pd

In [385]:
# Создайте матрицу A размерами 10 на 10 из упорядоченных целых чисел от 0 до 100
# hint: np.arange(100).reshape((10, 10))

A = np.arange(100).reshape(10,10)# Ваш код здесь
A

array([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
       [20, 21, 22, 23, 24, 25, 26, 27, 28, 29],
       [30, 31, 32, 33, 34, 35, 36, 37, 38, 39],
       [40, 41, 42, 43, 44, 45, 46, 47, 48, 49],
       [50, 51, 52, 53, 54, 55, 56, 57, 58, 59],
       [60, 61, 62, 63, 64, 65, 66, 67, 68, 69],
       [70, 71, 72, 73, 74, 75, 76, 77, 78, 79],
       [80, 81, 82, 83, 84, 85, 86, 87, 88, 89],
       [90, 91, 92, 93, 94, 95, 96, 97, 98, 99]])

In [386]:
# Посчитайте средние значения в матрице по столбцам

# Ваш код здесь
A_mean = np.aslist(map(np.mean, np.column_stack(A)));
A_mean

AttributeError: module 'numpy' has no attribute 'aslist'

In [387]:
# Транспонируйте матрицу A

# Ваш код здесь
A_t = A.transpose()

In [388]:
# Перемножьте поэлементно матрицу A и транспонированную матрицу A 

# Ваш код здесь
A_2 = A * A_t
print(A_2)

[[   0   10   40   90  160  250  360  490  640  810]
 [  10  121  252  403  574  765  976 1207 1458 1729]
 [  40  252  484  736 1008 1300 1612 1944 2296 2668]
 [  90  403  736 1089 1462 1855 2268 2701 3154 3627]
 [ 160  574 1008 1462 1936 2430 2944 3478 4032 4606]
 [ 250  765 1300 1855 2430 3025 3640 4275 4930 5605]
 [ 360  976 1612 2268 2944 3640 4356 5092 5848 6624]
 [ 490 1207 1944 2701 3478 4275 5092 5929 6786 7663]
 [ 640 1458 2296 3154 4032 4930 5848 6786 7744 8722]
 [ 810 1729 2668 3627 4606 5605 6624 7663 8722 9801]]


In [389]:
# Посчитайте matrix product (результат матричного умножения) матрицы A на саму себя и запишите в матрицу B
# https://en.wikipedia.org/wiki/Matrix_multiplication
# or https://docs.scipy.org/doc/numpy/reference/generated/numpy.dot.html

B = np.dot(A, A_t)# Ваш код здесь
B

array([[  285,   735,  1185,  1635,  2085,  2535,  2985,  3435,  3885,
         4335],
       [  735,  2185,  3635,  5085,  6535,  7985,  9435, 10885, 12335,
        13785],
       [ 1185,  3635,  6085,  8535, 10985, 13435, 15885, 18335, 20785,
        23235],
       [ 1635,  5085,  8535, 11985, 15435, 18885, 22335, 25785, 29235,
        32685],
       [ 2085,  6535, 10985, 15435, 19885, 24335, 28785, 33235, 37685,
        42135],
       [ 2535,  7985, 13435, 18885, 24335, 29785, 35235, 40685, 46135,
        51585],
       [ 2985,  9435, 15885, 22335, 28785, 35235, 41685, 48135, 54585,
        61035],
       [ 3435, 10885, 18335, 25785, 33235, 40685, 48135, 55585, 63035,
        70485],
       [ 3885, 12335, 20785, 29235, 37685, 46135, 54585, 63035, 71485,
        79935],
       [ 4335, 13785, 23235, 32685, 42135, 51585, 61035, 70485, 79935,
        89385]])

#### Не меняйте следующую ячейку. Просто запустите ее.

In [390]:
# Do not change me, just run
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
_dict_for_submission['B'] = transform_list(list(B.flatten()))
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

### Часть 2. Логистическая регрессия и kNN.

Будем работать с подвыборкой из [данных о типе лесного покрытия из репозитория UCI](http://archive.ics.uci.edu/ml/datasets/Covertype). Доступно 7 различных классов. Каждый объект описывается 54 признаками, 40 из которых являются бинарными. Описание данных доступно по ссылке, а так же в файле `covtype.info.txt`.

In [391]:
import pandas as pd

In [392]:
# If on colab, uncomment the following lines

# !wget "https://downloader.disk.yandex.com/disk/cd3ead013809a9da37434c792796d4b063dfc89571c68c273ca3788013b82bf8/5d865a2d/8bfV6Unxao6CYWcER5-G5fza25GZaFjE_xFvc0-ExCwlqmFjBf2bynRfBnL6ubJC8XhsoYHi7h9Nu6gfvS8wxg%3D%3D?uid=76549735&filename=forest_dataset.csv&disposition=attachment&hash=&limit=0&content_type=application%2Fvnd.ms-excel&owner_uid=76549735&fsize=1293856&hid=9b439c18e67923b45c3b3d8891bdf9ab&media_type=spreadsheet&tknv=v2&etag=45e530aa3c9796cb705d4e27be7a6e9c" -O forest_dataset.csv

In [393]:
all_data = pd.read_csv('forest_dataset.csv')

Выделим 20% данных в `delayed_data` в качестве выборки, на которой будет производиться финальное сравнение моделей (до конца ноутбука она более не используется), в `all_data` поместим оставшиеся 80%. Не будем перемешивать данные сознательно (для удобства проверки).

In [394]:
delayed_data = all_data[int(0.8 * len(all_data)):]
all_data = all_data[:int(0.8 * len(all_data))]

Выделим значения метки класса в переменную `labels`, признаковые описания в переменную `feature_matrix`. Так как данные числовые и не имеют пропусков, переведем их в `numpy`-формат с помощью метода `.values`.

In [395]:
labels = all_data[all_data.columns[-1]].values
feature_matrix = all_data[all_data.columns[:-1]].values

delayed_labels = delayed_data[delayed_data.columns[-1]].values
delayed_feature_matrix = delayed_data[delayed_data.columns[:-1]].values

#### 2.1 Двуклассовая классификация (10%)

Для начала будем работать только с двумя типами лесного покрытия. Обучите линейный классификатор для разделения двух классов. Данные разделите на train и test с помощью `train_test_split`.

In [396]:
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, accuracy_score

In [397]:
two_class_labels_indices = (labels == 1) + (labels == 2)

two_class_feature_matrix = feature_matrix[two_class_labels_indices]
two_class_labels = labels[two_class_labels_indices]

In [398]:
#?train_test_split

In [399]:
train_feature_matrix, test_feature_matrix, train_labels, test_labels = train_test_split(two_class_feature_matrix, two_class_labels, test_size=0.2)

Обучите логистическую регрессию. Оцените качество классификации на отложенной выборке с помощью `accuracy` и `f1_score` ([Wikipedia](https://en.wikipedia.org/wiki/F1_score)).

In [400]:
#?LogisticRegression
lr_clf = LogisticRegression()

In [401]:
#Попробуем сделать нормировку
from sklearn.preprocessing import minmax_scale

test_feature_matrix = minmax_scale(test_feature_matrix)
train_feature_matrix = minmax_scale(train_feature_matrix)

/home/serg/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64.
  warnings.warn(msg, DataConversionWarning)
/home/serg/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64.
  warnings.warn(msg, DataConversionWarning)


In [402]:
# Ваш код здесь
lr_clf.fit(train_feature_matrix, train_labels)
lr_predicted = lr_clf.predict(test_feature_matrix);

is_equal = list(map(lambda x,y: int(x == y) , lr_predicted, test_labels))
is_equal = np.asarray(is_equal)


#print("w = ", lr_clf.coef_)


/home/serg/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [403]:
accuracy = np.dot(is_equal, np.transpose(is_equal)) / len(is_equal)
print("Accuracy = " , accuracy)

#from sklearn
import sklearn.metrics as mes

print("SKLEARN accuracy = ", mes.accuracy_score(lr_predicted, test_labels), "SKLEARN f1_score = ", mes.f1_score(lr_predicted, test_labels))

Accuracy =  0.7186588921282799
SKLEARN accuracy =  0.7186588921282799 SKLEARN f1_score =  0.5987525987525988


#### 2.2 Многоклассовая классификация: логистическая регрессия (40%)

**Комментарий:**

*Логистическая регрессия позволяет решать и задачу многоклассовой классификации. Класс ``LogisticRegression`` позвляет это делать двумя способами:* 
- *1. Стандартный One vs Rest (т.е. каждый класс отделяется от всех других). Параметр `multi_class='ovr'`.*
- *2. Используюя кросс-энтропию (оценивается сразу вектор вероятностей принадлежности классам). Параметр `multi_class='multinomial'`.*

*По умолчанию будем использовать второй способ.*


Теперь будем работать со всеми 7 типами покрытия (данные уже находятся в переменных `feature_matrix` и `labels`, если Вы их не переопределили). Разделите выборку на обучающую и тестовую с помощью метода `train_test_split`, используйте значения параметров `test_size=0.2`, `random_state=42`. Обучите логистическую регрессию  на данном датасете.

In [405]:
lr_clf = LogisticRegression(multi_class='multinomial')
# Ваш код здесь
train_feature_mat, test_feature_mat, train_l, test_l = train_test_split(feature_matrix, labels, test_size=0.2, random_state=42)

С помощью 5-кратной кросс-валидации (`GridSearchCV`) подберите оптимальное значение параметров `С` и `penalty`. Для параметра `C` используйте набор значений `np.linspace(0.01, 1, 10)`, а для `penalty` - `['l1', 'l2']`. Критерий качества `scoring = 'accuracy'`.

In [406]:
# Ваш код здесь
from sklearn.model_selection import GridSearchCV
param_to_learn = {'C':np.linspace(0.01, 1, 10), 'penalty':}
clf = GridSearchCV(LogisticRegression(), )

Постройте графики средних значений критерия качества `accuracy` для типов регуляризации `l1` и `l2` в зависимости от коэффициента регуляризации `C`.

In [206]:
# Ваш код здесь

Проанализируйте полученные графики. Чему равны найденные оптимальные значения параметров `C` и `penalty`?

In [207]:
# Ваш код здесь

Используя найденные оптимальные значения `C` и `penalty`, обучите логистическую регрессию на обучающей выборке (train) и вычислите вероятности принадлежности к классам для тестовой (test)  выборки (`best_lr_clf.predict_proba`).

In [208]:
best_lr_clf = LogisticRegression(<your_best_params_here>)

# Ваш код здесь

SyntaxError: invalid syntax (<ipython-input-208-227cb0e1fee6>, line 1)

Для полученной матрицы вычислите усредненные по всем объектам из тестовой выборки значения вероятностей принадлежности к классам. Сравните ее оценкой вероятности принадлежности к классу на основе частот меток классов в обучающей выборке.

In [ ]:
# Ваш код здесь

#### 2.3 Многоклассовая классификация: kNN (30%).

Вспомним первое занятие. Обучите на том же датасете `KNeighborsClassifier` из `sklearn`.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

# Ваш код здесь

Попробуйте использовать различные стратегии вычисления весов: `[‘uniform’, ‘distance’]`. Выберите одно из них самостоятельно. Чем обоснован Ваш выбор?

Переберите по сетке от `1` до `10` параметр числа соседей. Покажите на графике, как менялись значения критерия качества `accuracy` при изменении числа соседей.

In [ ]:
# Ваш код здесь

Используя найденное оптимальное число соседей, вычислите вероятности принадлежности к классам для тестовой выборки (`.predict_proba`).

In [ ]:
# Ваш код здесь

Создайте объект `best_knn_clf`, который представляет собой лучший классификатор.

In [ ]:
# Hint: для этого достаточно указать стратегию вычисления весов и число соседей в конструкторе

best_knn_clf = KNeighborsClassifier(n_neighbors=<число соседей>, weights=<'uniform' или 'distance'>)

#### 2.4 Сравнение результатов логистической регрессии и kNN (10%).

В реальной жизни новые данные становятся доступны с течением времени. Сравните качество Вашей лучшей модели логистической регресии `best_lr_clf` и лучшего kNN `best_knn_clf` на отложенной выборке `delayed_data`. Какая из моделей показала лучший результат?

In [ ]:
# Ваш код здесь

#### Не меняйте следующую ячейку. Запустите ее для генерации `submission` файла.

In [ ]:
# Do not change me, just run
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
_dict_for_submission['best_lr_clf_delayed'] = transform_list(list(best_lr_clf.predict(delayed_feature_matrix)))
_dict_for_submission['best_knn_clf_delayed'] = transform_list(list(best_knn_clf.predict(delayed_feature_matrix)))

import json
with open('submission_hw1_basic.json', 'w') as iofile:
    json.dump(_dict_for_submission, iofile)
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

Данный файл необходимо будет прикрепить к Google-форме для сдачи домашнего задания. __Для идентификации используется e-mail, который вы указали при регистрации на курс. Проверка производится автоматически. Если вы подпишитесь другим e-mail'ом, задание просто не будет засчитано.__

На этом задание завершено. Мы будем рады, если вы оставите обратную связь по данному заданию [по ссылке](https://forms.gle/2qaf14ky3J6HZfe48). Она анонимна ;)